In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def compute_derivs(k, m, g, l0, X):
    r = X[0]
    theta = X[1]
    dr = X[2]
    dtheta = X[3]
    ddr = (r*dtheta*dtheta)+(g*np.cos(theta))+(k/m)*(l0-r)
    ddtheta = -((g*np.sin(theta))+(2*dr*dtheta))/r
    return np.array([dr, dtheta, ddr, ddtheta])

def do_rk4(r0, theta0, dr0, dtheta0, t, h, k, m, g, l0):
    X = np.zeros((len(t), 4))
    
    X[0][0] = r0
    X[0][1] = theta0
    X[0][2] = dr0
    X[0][3] = dtheta0
    
    for i in range(1, len(t)):
        k1 = compute_derivs(k, m, g, l0, X[i-1])
        k2 = compute_derivs(k, m, g, l0, X[i-1]+((h*k1)/2))
        k3 = compute_derivs(k, m, g, l0, X[i-1]+((h*k2)/2))
        k4 = compute_derivs(k, m, g, l0, X[i-1]+(h*k3))
        X[i] = X[i-1]+(k1+k2+k2+k3+k3+k4)*(h/6)
        
    return X

def compute_total_energy(X, k, m, g, l0):
    x = X.T
    v2 = np.multiply(x[0]**2, x[3]**2)+(x[2]**2)
    ke = 0.5*m*v2
    pe = (-m*g*x[0]*np.cos(x[1]))+(0.5*k*((l0-x[0])**2))
    return ke,pe

In [8]:
g = 9.81
k = 1
m = 0.6*k
l0 = 1
r = 10
theta = 0 
dr = -2
dtheta = 100
h = 0.0005
t = np.arange(0, 20+h, h)
V = do_rk4(r, theta, dr, dtheta, t, h, k, m, g, l0)
ke, pe = compute_total_energy(V, k, m, g, l0)
te = ke+pe

In [9]:
%matplotlib qt
plt.figure()
plt.plot(t, V.T[0])
plt.show()
plt.figure()
plt.plot(t, V.T[1])
plt.show()
plt.figure()
plt.plot(t, te)
plt.show()

In [10]:
X = V.T[0]*np.sin(V.T[1])
Y = -V.T[0]*np.cos(V.T[1])
f = 20


fig, ax = plt.subplots()
ax.set_xlim(min(X)-10, max(X)+10)
ax.set_ylim(min(Y)-5, 10+1)
ax.scatter(0, 0, c='red', s=80, label='Fixed point (0, 0)')
scat = ax.scatter([], [], s=[], c='blue')
trail_line, = ax.plot([], [], 'b--', linewidth=1, alpha=0.6)\

coord_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
r_text = ax.text(0.02, 0.88, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
theta_text = ax.text(0.02, 0.81, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
energy_text = ax.text(0.02, 0.73, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
time_text = ax.text(0.8, 0.95, '', transform=ax.transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
trail_x = []
trail_y = []

def init():
    trail_x.clear()
    trail_y.clear()
    trail_line.set_data([], [])
    scat.set_offsets([[X[0], Y[0]]])
    scat.set_sizes([10])
    coord_text.set_text(f'Coordinates: ({X[0]:.2f}, {Y[0]:.2f})')
    r_text.set_text(f'R: ({V.T[0][0]:.2f})')
    theta_text.set_text(f'Theta: ({V.T[1][0]:.2f})')
    energy_text.set_text(f'Energy: ({te[0]:.2f})')
    time_text.set_text(f'Time: ({t[0]:.2f})')
    return scat, coord_text, r_text, theta_text, energy_text, time_text

def update(frame):
    x, y = X[10*frame], Y[f*frame]
    trail_x.append(x)
    trail_y.append(y)
    scat.set_offsets([[x, y]])
    scat.set_sizes([10])
    trail_line.set_data(trail_x, trail_y)
    coord_text.set_text(f'Coordinates: ({x:.2f}, {y:.2f})')
    r_text.set_text(f'R: ({V.T[0][f*frame]:.2f})')
    theta_text.set_text(f'Theta: ({V.T[1][f*frame]:.2f})')
    energy_text.set_text(f'Energy: ({te[f*frame]:.2f})')
    time_text.set_text(f'Time: ({t[f*frame]:.2f})')
    return scat, trail_line, coord_text, r_text, theta_text, energy_text, time_text

ani = animation.FuncAnimation(fig, update, frames=(len(X)//10),
                              init_func=init, interval=1)

plt.show()
